# Lumber Drying Time

[Source of Red Oak equations](https://en.wikipedia.org/wiki/Wood_drying)

8.07131 	1730.63 	233.426 	1 	99
T(°C) = (T(°F) - 32) × 5/9


![Formula](https://upload.wikimedia.org/math/e/9/5/e95d69b400fdbdc5e8af6cc7bb6e7395.png)
![Time Constant](https://upload.wikimedia.org/math/a/8/d/a8df91af00c8291dd3b4d399e35f41ae.png)

In [1]:
def p_sat(T):
    """
    Saturation vapor pressure of water - T in degress F
    See: https://en.wikipedia.org/wiki/Vapour_pressure_of_water#Approximation_formula
    """
    A = 8.07131
    B = 1730.63
    C = 233.426
    T_C = (T - 32) * 5/9
    p = 10**(A - B/(C+T_C))
    return p

p_sat(150)

191.8194885201814

In [2]:
def τ(L,T):
    "Time Constant"
    a = 0.0575
    b = 0.00142
    n = 1.52 
    t = L**n / (a+b*p_sat(T))
    return t

def M_eq(h, T=72):
    """
    Equlibrium Moisture Content 
    
    Temperature (F) 
    Relative Humidity (h)
    See: https://en.wikipedia.org/wiki/Equilibrium_moisture_content
    """
    W = 330+0.452*T+0.00415 * T**2
    k = 0.791+4.63*10**-4*T-8.44*10**-7*T**2
    k1 = 6.34+7.75*10**-4*T-9.35*10**-5*T**2
    k2 = 1.09+2.84*10**-2*T-9.04*10**-5*T**2
    a = k1*k2*k**2*h**2
    b = k1*k*h
    M_e = 1800/W *(k*h/(1-k*h)+(b+2*a)/(1+b+a))
    return M_e

![Moisture W](https://upload.wikimedia.org/math/4/4/e/44e64c8f85e7b50aeb2c671c3cc7e1c1.png)
![Moisture k](https://upload.wikimedia.org/math/2/9/7/297e89503659c6bcb292e67df2b243b7.png)
![Moisture k1](https://upload.wikimedia.org/math/e/e/e/eeeafc23f767fd24808b1e9f5cafd6fc.png)
![Moisture k2](https://upload.wikimedia.org/math/8/4/6/84664131ca2da179d131756dbfd0da4a.png)

![Equilibrium Moisture](https://upload.wikimedia.org/math/4/d/8/4d80598eabc53e943dc7d05d3b417abd.png)

![Drying Time](https://upload.wikimedia.org/math/c/b/0/cb0d2622cbbf62e49af6511c4296ea7c.png)

## Drying Time Calculations

In [3]:
import numpy as np

def t(M, M_0, h=0.72, L=1.5, T=72):
    """
    Drying Time in days
    
    M - Desired Moisture Content
    M_0 - Starting Moisture Content
    h - relative humidity (72% average annual RH for Gainesville FL)
    L - Smallest dimension(inches)
    T - Temperature (F)
    """
    M_e = M_eq(h,T)
    if M <= M_e:
        raise ValueError("M must above the Equilibrium Moisture Content of {:.3f}".format(M_e))
    t_ = -τ(L,T)*np.log((M-M_e)/(M_0-M_e))
    print("""\tA {} inches thick board of Red Oak with an initial moisture 
    content of {}% will dry to {}% M.C. in {:.1f} Days if placed in a  
    constant {:.0%} Relative Humidity evironment at {} F.""".format(L,M_0,M,t_,h,T))
    return t_

In [4]:
t(14,100)

	A 1.5 inches thick board of Red Oak with an initial moisture 
    content of 100% will dry to 14% M.C. in 113.4 Days if placed in a  
    constant 72% Relative Humidity evironment at 72 F.


113.35498604905855